In [1]:
import spacy
from spacy.tokenizer import Tokenizer
import nltk.data
import nltk
from nltk import word_tokenize
import re
from nltk import FreqDist
import pandas as pd
import re
import csv
import pickle
import itertools
import glob
#!pip install duplicates
#!pip install iteration-utilities
from iteration_utilities import duplicates
from iteration_utilities import unique_everseen

In [2]:
#Classe de cores para destacar certas partes dos prints
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

### Carregando dados de texto a serem rotulados

In [38]:
#Carregando os mesmos textos que foram utilizados para a rotulação manual

dados_De_Texto_Para_Rotular = []
with open("C:\\Users\prisc\\Downloads\\PROJETO_MESTRADO\\Atual_Codigo_Para_Rotulacao_SemiAutomatica\\Dados_Utilizados_Para_RotulacaoManual.txt", "r",encoding='utf-8') as fp:
    list_Lines = fp.readlines()
    for line in list_Lines:
        dados_De_Texto_Para_Rotular.append(line.replace('\n',''))

for item in dados_De_Texto_Para_Rotular:
    print(item)

id: 0
NOME DA CACHAÇA: Cachaça Três Coronéis e Cachaça Três Coronéis
PREÇO: R$ 35,00
DESCRIÇÃO DA CACHAÇA: Cachaça Três Coronéis 700ml
Volume: 750ml
Graduação Alcoólica: 39,2%
RECIPIENTE DE ARMAZENAMENTO: Tipo Barril: Carvalho
Tipo: Envelhecida
Produtor: Glauri Ind Com. Ltda
Local de Produção: Boituva / São Paulo
id: 1
NOME DA CACHAÇA: Cachaça Princesa Isabel Jaqueira Ouro
PREÇO: R$ 85,00
DESCRIÇÃO DA CACHAÇA: Na beira do Rio Doce cultivamos cacau na forma mais tradicional e sustentável.
Lá, um certo dia no ano de 2015, uma frondosa jaqueira secular tombou durante um temporal, foi quando nosso patriarca teve essa grande ideia.
Recolheu o tronco caído e levou para que nosso tanoeiro fizesse barris com a madeira.
De volta à fazenda Tupã eles foram enchidos de cachaça Princesa Isabel e o resultado foi essa surpreendente e singular bebida.
Uma bela cachaça, com coloração amarelho esverdeado, límpida, brilhante a muito encorpada.
No olfato percebemos pouco álcool, destacando aromas adocicad

PREÇO: R$ 399,99
DESCRIÇÃO DA CACHAÇA: Visual: - Cristalino, Oleosidade e com lágrimas caindo no tempo correto.
Aroma: - Aroma bem adocicado herbal com notas marcantes de especiarias como canela e baunilha.
Boca: Macia aveludada com uma leve e interessante apimentadinha e baixíssima acidez.
Retrogosto: bem amadeirado persistente e alongado.
Harmonização: Cachaça perfeita para adicionar gelo e acompanhar um chocolate meio amargo, e celebrar com os amigos.
Origem: Jandaia do Sul - PR Madeira:Amburana (Imburana).
Envelhecimento: Descansada em barril de Amburana (Imburana).
Graduação Alcoólica: 40% Volume: Garrafa de vidro padronizada de 500ml
Volume: 750ml
id: 112
NOME DA CACHAÇA: Cachaça Estância Moretti 4 Madeiras 500ml
PREÇO: R$ 399,99
Volume: 500ml
id: 113
NOME DA CACHAÇA: Cachaça Remedin Ouro 500ml
PREÇO: R$ 59,99
DESCRIÇÃO DA CACHAÇA: Produzida por um casal de mineiros que reside há vinte anos em Brasília, a Remedin guarda todas as tradições da produção artesanal, como o alambique d

id: 187
NOME DA CACHAÇA: Cachaça Premium Seleta Antonio Rodrigues 700 ml
VOLUME: 700ml
DESCRIÇÃO DA CACHAÇA: Cachaça Premium Seleta Antonio Rodrigues 700 ml
Antônio Rodrigues é uma bebida única, a verdadeira joia líquida.
Uma série limitada, especial para os apreciadores da cachaça típica de Minas.
Envelhecida em carvalho francês, seu aroma é encorpado e o sabor inesquecível.
Límpida, translúcida, viscosa e brilhante.
A Antônio Rodrigues foi criada para ser uma cachaça tradicional, de cor forte e escura, resultado de toda a experiência e excelência da Seleta na produção da bebida.
A “joia líquida”, como ficou conhecida, é sinônimo de sabor agradável e de boa intensidade.
Como Servir:A joia líquida da Seleta é uma bebida saborosa, complexa e feita para um paladar diferenciado.
Experimente-a pura, à temperatura ambiente.
Devido à sua intensidade, a cachaça Antônio Rodrigues pode ser também consumida na versão “on the rocks”, tomando bastante cuidado com o tipo de gelo usado, evitando dil

id: 252
NOME DA CACHAÇA: Cachaça Cachoeira Copa especial 1000 ml
PREÇO: R$ 59,90
Volume: 1000 ml
DESCRIÇÃO DA CACHAÇA: Cachaça Cachoeira Copa especial 1000 mlProduzida na cidade de Salinas (MG)Armazenado em tonéis de BalsamoConteúdo : 1000 mlGarrafa : Especial copa do mundo transparentePrazo de Validade : IndeterminadoGraduação : 48 %Evite o consumo Excessivo de Álcool
Grad. Alcool: 48%
id: 253
NOME DA CACHAÇA: Miniatura Nova Aliança Bálsamo
PREÇO: R$ 12,90
Volume: 50 ml
DESCRIÇÃO DA CACHAÇA: Cachaça Nova Aliança
           A Cachaça Nova Aliança foi criada pensando estabelecer relação de confiança
e o reconhecimento do consumidor junto ao produtor industria, Comercio e Exportação de Cachaça
Beija - Flor Ltda, empresa familiar de Salinas que está no mercado desde de 1988, que tamnbem produz a famosa e tradicional Indaiazinha.
A Cachaça Nova Aliança e comercializada em três versões de armazenamento em madeiras de Bálsamo,
umburana e jequitibá.
Tais madeiras conferem a bebida paladar inc

Validade: Indeterminada
Produzida: Artesanalmente
Produzida em: Artesanalmente
id: 311
NOME DA CACHAÇA: Cachaça Evelhecida Jequitibá Da Torre 500 ML
PREÇO: R$43,90
DESCRIÇÃO DA CACHAÇA: Cachaça Evelhecida Jequitibá Da Torre 500 ML ORIGEM: Produzido em Amparo - São Paulo Alambique da Torre Cachaça da Torre Envelhecida em Barril de Jequitibá Rosa por 18 meses.
Grad.
alc: 40% Volume: 500 ml.
A Cachaça da Torre descansada foi premiada em 3o lugar (medalha de bronze) em 2008 e em 2o lugar (medalha de prata) em 2012, no Concurso de Avaliação de Cachaças, durante o Brazilian Meeting on Chemistry of Food and Beverages, supervisionado pelo Instituto de Química de São Carlos (IQSC-USP).
Volume: 500 ML
LOCAL DE PRODUÇÃO: Produzido em Amparo - São Paulo
id: 312
NOME DA CACHAÇA: Cachaça Rio do Engenho Reserva Umburana, Itiúba, Louro Canela e Balsamo 670 ml
PREÇO: R$89,90
DESCRIÇÃO DA CACHAÇA: Cachaça Rio do Engenho Reserva Umburana, Itiúba, Louro Canela e Balsamo 670 ml www.cachacaepresente.com.br 

...

O Produtor
Criada em 2015 pelos Mestres Cachaciers Renato Bittencourt e Arnaldo Ribeiro, Antonieta é produzida na fazenda Taverna Real, em Itaverava, Minas Gerais.
A Taverna Real situa-se na antiga Estrada Real, rota designada pela coroa portuguesa em meados do Século XVII para escoar o ouro e os diamantes mineiros.
A Cachaça Antonieta é produzida no município de Ituverava, em Minas Gerais.
A cadeia produtiva segue um rigoroso processo de controle, desde o cuidado com o solo onde a cana-de-açúcar é plantada, passando pela destilação feita em pequena escala em alambiques de cobre, até o envase em garrafas importadas da França.
Harmonize ao seu gosto!
Volume: 750 ml
LOCAL DE PRODUÇÃO: Minas Gerais - Brasil
Produtor: Antonieta
GRADUAÇÃO ALCOÓLICA: 37%
Madeira: Amburana
VISÃO: caramelo
OLFATO: baunilha, frutas-secas, mel
PALADAR: adocicado, madeira
id: 764
NOME DA CACHAÇA: Cachaça Antonieta Jequitibá 750ml
PREÇO: R$ 80,00
DESCRIÇÃO DA CACHAÇA: Cachaça Antonieta Jequitibá 750ml
A Cachaça An

A cachaça Saliníssima Prata é produzida na Fazenda Matrona, produtora de cachaças desde 1955, localizada em Salinas – Minas Gerais, região de denominação de origem (DO) composta pela cidade de Salinas e outros cinco municípios vizinhos.
Ela é a versão pura, que não passa da madeira, da Saliníssima – sua companheira Saliníssima Ouro passa por barris de bálsamo, madeira tipicamente usada na região de Salinas para armazenamento da bebida.
Natique, criadora das marcas de cachaça Santo Grau, Espírito de Minas e Saliníssima
A Natique é uma empresa com mais de 20 anos trabalhando o conceito de cachaça artesanal no mercado brasileiro.
Com um portfólio com mais de 15 marcas, produzidas em diversos cantos do Brasil, é a responsável pelas marcas Santo Grau, Espírito de Minas e Saliníssima.
Idealizada por Paulo Pellotas e Duique Munhoz, a Natique trabalha com produtores tradicionais do Rio de Janeiro (Santo Grau Paraty), Minas Gerais (Santo Grau Século XVIII, Santo Grau Coronel Xavier Chaves e Sa

Origem: Pataty/RJ
GRADUAÇÃO ALCOÓLICA: 27%
TIPO: bebida mista
LOCAL DE PRODUÇÃO: Pataty/RJ
id: 842
NOME DA CACHAÇA: Cachaca Paratiana Ouro Ceramica 500ml
PREÇO: R$ 105,90
VOLUME: 500ml
DESCRIÇÃO DA CACHAÇA: 500ml
42%
carvalho
2 anos
ouro
Paraty/RJ
INFORMAÇÕESOES TÉCNICAS: Volume
Graduação Alcoólica
Madeira
Tempo de Armazenamento
Tipo
Origem
Madeira: Carvalho
Envelhecimento: 2 anos
Teor Alcoólico: 42%
Volume: 700ml
Tipo: Ouro
Origem: Paraty/RJ
Envelhecida de 1 a 2 anos em
História da Paratiana
Em Janeiro da 1996, Carlos José Gama Miranda ( Casé ) junto com Dom João Orleans e Bragança, iniciam no bairro do Corisco uma sociedade na produção da Cachaça Maré Alta.
Em Abril de 1999 Casé inicia uma nova sociedade com seu irmão Paulo Eduardo Gama Miranda e o desenvolvimento de um novo Alambique, em um antigo casarão, cercado pela mata atlântica ladeado de cachoeira no bairro Pedra Branca, aliando ao modo de fabricação da Cachaça de Paraty ele realiza seu antigo sonho, e lança a Cachaça Paratia

Tipo
Origem
TEXTO DESCRITIVO: Armazenamento
: Carvalho
 Volume: 500ml
Tipo: Premium
Origem: Araguari / MG
Produzida artesmalmente e envelhecida por 5 anos em tonéis de Carvalho Americano, o sabor dessa madeira incorporado ao da cana de açúcar confere a essa cachaça uma sensação única, combina com todo tipo de creme e molhos leves, Pode ser consumida pura ou com gelo.
TIPO: premium
LOCAL DE PRODUÇÃO: Araguari / MG
id: 877
NOME DA CACHAÇA: Cachaça Arraial Velho Prata 965ml
PREÇO: R$ 41,90
VOLUME: 965ml
DESCRIÇÃO DA CACHAÇA: 965ml
ouro
Araguari / MG
INFORMAÇÕESOES TÉCNICAS: Volume
Tipo
Origem
Volume: 700ml
Tipo: Ouro
Origem: Araguari / MG
TIPO: ouro
LOCAL DE PRODUÇÃO: Araguari / MG
id: 878
NOME DA CACHAÇA: Cachaça Arraial Velho Ouro Cerejeira 965ml
PREÇO: R$ 41,90
VOLUME: 965ml
DESCRIÇÃO DA CACHAÇA: 965ml
ouro
Araguari / MG
INFORMAÇÕESOES TÉCNICAS: Volume
Tipo
Origem
TEXTO DESCRITIVO: Madeira
Volume: 965ml
Tipo: Ouro
Origem: Araguari / MG
TIPO: ouro
LOCAL DE PRODUÇÃO: Araguari / MG
id: 87

PREÇO: R$ 96,00
DESCRIÇÃO DA CACHAÇA: Cachaça Terra de Ouro balsamo 700ml procelanaA Cachaça Terra de Ouro é produzida na cidade mineira de Salinas, região internacionalmente reconhecida por abrigar as melhores cachaças existentes no mercado mundial.
Sua elaboração é totalmente artesanal, dotada de segredos e técnicas de gente que vive em meio à produção de cachaça desde que nasceu, conservando a essência das cachaças de Salinas.Madeira: BálsamoGrad.
Alcoólica: 42%​Volume: 700 mlTempo de Envelhecimento: 2 anosOrigem: Salinas/MGColoração: DouradaAnálise: Sabor suave.
Aroma misto de madeira (bálsamo) com a pura cachaça artesanal.
Volume: 700ml
id: 955
NOME DA CACHAÇA: Cachaça Terra de Ouro balsamo
PREÇO: R$ 49,90
DESCRIÇÃO DA CACHAÇA: Cachaça Terra de Ouro balsamo 700ml A Cachaça Terra de Ouro é produzida na cidade mineira de Salinas, região internacionalmente reconhecida por abrigar as melhores cachaças existentes no mercado mundial.
Sua elaboração é totalmente artesanal, dotada de segr

### Convertendo os textos para letras minusculas

In [4]:
#Convertendo todo o texto para letras minusculas

#dados_De_Texto_Para_Rotular = list(map(str.lower,dados_De_Texto_Para_Rotular))
#dados_De_Texto_Para_Rotular =  list(set(dados_De_Texto_Para_Rotular))
#print(color.BOLD+'Tamanho do dataset após a remoção dos dados repetidos:'+color.END, len(dados_De_Texto_Para_Rotular))

 #list_Lines = fp.readlines()
        #for line in list_Lines:
            #dados_De_Texto_Para_Rotular.append(line.replace('\n',''))

### Carregando os dicionários que contém as entidades versus categorias

In [23]:
# 1 - Cria uma lista com os endereços dos arquivos no formato de dicionario. Dicionarios possuem que vários elementos...
# a chave é a entidade (texto) e o valor a categoria desse elemento
# 2 - Um laço for é utilizado para percorrer essa lista de endereços
# 3 - E a cada volta do laço um endereço é passado para o método with open() e, o arquivo binário do referido endereço é aberto
# 4 - E o conteúdo do arquivo (um diciário com vários elementos) é inserido na lista de dicionarios.
# Desta maneira teremos uma lista composta por vários dicionarios. E vários dicionários que possuem vários itens. E os itens...
# são as entidades versus categorias, que o algoritmos de rotulação semiautomatica precisa para rotular os textos.

files_Dicionario = glob.glob ('C:\\Users\\prisc\\Downloads\\PROJETO_MESTRADO\\Entidades_E_Categorias_Para_Usar_No_Algoritmo_De_RotulacaoSemiAutomatica\*.txt')

lista_Dicionarios = []

for index, file in enumerate(files_Dicionario):
    with open(file, "rb") as fp:
        lista_Dicionarios.append(pickle.load(fp))

print("Total de itens - Entidades x Categorias:", len(lista_Dicionarios[0]))
len(lista_Dicionarios)

Total de itens - Entidades x Categorias: 2906


1

In [24]:
# 5 - Cria um único dicionario com todos os dicionarios que estam em diferentes posições da lista de dicionarios.

dicionario_EntidadeCategoria = {}

for item in lista_Dicionarios :
    dicionario_EntidadeCategoria.update(item)
    #print(dicionario_EntidadeCategoria )
    #print('---------------------------------------------------------------------------------------------------------------')
print("Total de itens - Entidades x Categorias:", len(dicionario_EntidadeCategoria))

Total de itens - Entidades x Categorias: 2906


In [7]:
# Convertendo os textos do diconario (campo Key) para letras minusculas
#dicionario_EntidadeCategoria = {k.lower(): v for k, v in dicionario_EntidadeCategoria.items()}

#OBS: a quantidade de dados do dicionario diminui porque a propria inserção dos dados de texto em minusculo remove os item 
# duplicados.
#len(dicionario_EntidadeCategoria)

In [32]:
# alterando o texto que no dicionario que está diferente

dicionario_EntidadeCategoria = { k.replace('Cachaça Doce Paixão.', 'Cachaça Doce Paixão'): v for k, v in dicionario_EntidadeCategoria.items()}
dicionario_EntidadeCategoria

{'Cachaça Três Coronéis': 'NOME_CACHACA',
 'Cachaça Princesa Isabel Jaqueira Ouro': 'NOME_CACHACA',
 'Cachaça Colombina Chita': 'NOME_CACHACA',
 'Cachaça Colombina 3 anos': 'NOME_CACHACA',
 'Cachaça Colombina 10 anos': 'NOME_CACHACA',
 'Cachaça Babuxca Tangerina com Pimenta': 'NOME_CACHACA',
 'Cachaça Babuxca Frutas Vermelhas': 'NOME_CACHACA',
 'CACHAÇA SERRA MORENA OURO': 'NOME_CACHACA',
 'Cachaça da Tulha Prata': 'NOME_CACHACA',
 'CACHAÇA GOGÓ DA EMA SUBLIME': 'NOME_CACHACA',
 'Cachaça Maribondo': 'NOME_CACHACA',
 'Porto Estrela Ouro 1 Litro': 'NOME_CACHACA',
 'Santo Mel': 'NOME_CACHACA',
 'João Mendes Prata': 'NOME_CACHACA',
 'Gouveia Brasil MEI - Cachaça Limao e Mel': 'NOME_CACHACA',
 'Cachaça Reserva do Gerente Carvalho 5 anos': 'NOME_CACHACA',
 'Cachaça Reserva do Gerente Prata': 'NOME_CACHACA',
 'Cachaça Maria da Cruz': 'NOME_CACHACA',
 'Cachaça Taverna de Minas Carvalho': 'NOME_CACHACA',
 'Taverna de Minas Amburana': 'NOME_CACHACA',
 'Cachaça Taverna de Minas Jequitibá': 'NOME_

In [31]:
from requests.structures import CaseInsensitiveDict

dicionario_EntidadeCategoria = CaseInsensitiveDict(dicionario_EntidadeCategoria)
#dicionario_EntidadeCategoria

In [35]:
# Coloca os itens do dicionario em uma listas, assim cada item do dicionario vira uma tupla

listaDicionariosOrdenadosComEntidadesJuntos = list(dicionario_EntidadeCategoria.items())
#listaDicionariosOrdenadosComEntidadesJuntos.sort(key=len x: x[0])
#listaDicionariosOrdenadosComEntidadesJuntos

In [20]:
# O código cria uma lista a partir dos termos que compõem as chaves (keys) do dicionario 'dicionario_EntidadeCategoria'...
# Isso é necessário para que o código da linha abaixo funcione.

list_termos_Equivalentes_A_Entidade = list(dicionario_EntidadeCategoria)
len(list_termos_Equivalentes_A_Entidade)


# Modifica o caracter '|' por '||', porque logo mais a frente, haverá uma celula com código onde 
# essa lista com as entidades será transforma em strings sepradas por '|', para evitar erro na parte do código de rotulaçaõ
#semiautomatica é necessario substituir '|' por '||'

termos_Equivalentes_A_Entidade = []

for item in list_termos_Equivalentes_A_Entidade:
    boolean = re.search("\|", item)
    if boolean:
        item_Da_Lista = re.sub('\|', '_', item, count=1)
        termos_Equivalentes_A_Entidade.append(item_Da_Lista)
    else:
        termos_Equivalentes_A_Entidade.append(item)
        
termos_Equivalentes_A_Entidade

['Cachaça Três Coronéis',
 'Cachaça Princesa Isabel Jaqueira Ouro',
 'Cachaça Colombina Chita',
 'Cachaça Colombina 3 anos',
 'Cachaça Colombina 10 anos',
 'Cachaça Babuxca Tangerina com Pimenta',
 'Cachaça Babuxca Frutas Vermelhas',
 'CACHAÇA SERRA MORENA OURO',
 'Cachaça da Tulha Prata',
 'CACHAÇA GOGÓ DA EMA SUBLIME',
 'Cachaça Maribondo',
 'Porto Estrela Ouro 1 Litro',
 'Santo Mel',
 'João Mendes Prata',
 'Gouveia Brasil MEI - Cachaça Limao e Mel',
 'Cachaça Reserva do Gerente Carvalho 5 anos',
 'Cachaça Reserva do Gerente Prata',
 'Cachaça Maria da Cruz',
 'Cachaça Taverna de Minas Carvalho',
 'Taverna de Minas Amburana',
 'Cachaça Taverna de Minas Jequitibá',
 'Cachaça Weber Haus Leandro Batista',
 'Cachaça Gogó da Ema Rock N´Roll Caveira Bálsamo',
 'Lançamento Weber Haus ( 7 Madeiras + Umas e Outras + Leandro Batista )',
 'Cachaça Umas e Outras',
 'AGUARDENTE COMPOSTA COQUEIRO AZULADA',
 'Coqueiro Gabriela',
 'Cachaça Fabulosa',
 'Cachaça Hanavilhana Ouro',
 'Cachaça Salicana Ou

In [19]:
lista_Com_NomesDeBebidas_Ordenados = sorted(termos_Equivalentes_A_Entidade,key=len,reverse=True)
len(lista_Com_NomesDeBebidas_Ordenados)

for item in lista_Com_NomesDeBebidas_Ordenados:
    print(item)

Vanderley Azevedo 12 anos + Vanderley Azevedo 6 anos + Vanderley Azevedo Prata + Vanderley Azevedo 4 anos _ Empório Cachaça Canela-de-Ema
Cachaça Artesanal Mineira João Plenário Blend 700 ml - Cachaça é Presente - Caipirinha Morena
Cachaça Havana Edição Comemorativa 75 anos 750 ml com Certificado N. 253 Tradicional
Cachaça Cambéba Extra Premium - 7 anos -  700 ml  _ Empório Cachaça Canela-de-Ema
Cachaça Espírito de Minas Coleção Artistas Brasileiros Por Elifas Andreato 700ml
Cachaça Trindade Artesanal Platinum Amendoim Porcelana 700 ml cachaça é Presente
Cachaça Mazzaropi Envelhecida Carvalho Francês 4 Anos 700 ml Cachaça e Presente
Cachaça Cambéba Extra Premium - 5 anos - 700ml  _ Empório Cachaça Canela-de-Ema
Cachaça Espírito de Minas Coleção Artistas Brasileiros Por Claudio Tozzi 700ml
Cachaça Rio do Engenho Reserva Umburana, Itiúba, Louro Canela e Balsamo 670 ml
Cachaça Casa Bucco Extra Premium 6 Anos Garrafa Especial Personalizada 700ml
Cachaça Oriente de Minas - Carvalho  - 670ml

Cachaça Prosa Mineira Tradicional
CACHAÇA MAGNIFICA RESERVA SOLEIRA
Aguardente Flor da Montanha Prata
Cachaça Sapucaia Florida Amendoim
Cachaça Weber Haus Playboy Silver
Cachaça Sapucaia Florida Carvalho
Cachaça Weber Haus Prata Orgânica
Cachaça Sapucaia Florida Superior
Cachaça Weber Haus Premium Pocket
Aguardente de Cana Velha Januária
Cachaça Engenho D'Ouro Prata Mini
Cachaça Harmonie Schnaps Amburana
Cachaça Indiazinha Flecha de Rubi
Aguardente de Cana Catuy Morena -
Cachaça Companheira Branca 670 ml
Cachaça Salinas Cristalina 600 ML
Cachaça 1000 Montes Carvalho 3AOB
Cachaça Carvalheira BRASIL Branca
Cachaça Casa Bucco Prata Orgânica
Cachaça Chico Mineiro Ouro 01 lt.
Cachaça Du Brejo Sassafrás Canela
Cachaça Mato Dentro Ouro New Beat
Cachaça Mato Dentro Premium Prata
Cachaça Se Sobrá Autentica Jatobá
Cachaça Costa Rica Carvalho 670ml
Cachaça Weber Haus Amburana 700ml
Cachaça Velha Januaria Ouro 600ml
Cachaça Velha Januaria Ouro 670ml
Cachaça Tiúba de Minas Ouro 600ml
Cachaça Asa Br

Cachaça Peruaçu Ouro 670ml
Cachaça Paraiso Ouro 700ml
Cachaça Moicana Ouro 670ml
Cachaça Lapinga Ouro 670ml
Cachaça Jacuba Prata 700ml
Cachaça Jornada Ouro 670ml
Cachaça Gaivota Ouro 600ml
Cachaça General Ouro 900ml
Cachaça Frazão Prata 670ml
Cachaça Fagulha Ouro 860ml
Cachaça Duvido Prata 600ml
Cachaça Bocaina Ouro 980ml
Cachaça Bocaina Ouro 670ml
Cachaça Artista Ouro 670ml
Cachaça Japi Reserva 750ml
Cachaça Gavena Prata 750ml
Cachaça Tabocal Ouro 700ml
Cachaça Seleta Ouro 1000ml
Cachaça Serrote Ouro 750ml
Cachaça Yaguara Ouro 750ml
Cachaça Ouro 1 Prata 750ml
Cachaça Ouro 1 Velha 750ml
Cachaça Mucambo Ouro 700ml
Cachaça Gravata Ouro 700ml
Cachaça Formosa Ouro 700ml
Cachaça Germana Soul 700ml
Companheira de Bolso Prata
Cachaça Tabúa Prata 670 ml
Terra de Ouro Prata 600 ml
Cachaça Hanavilhana 600 ml
Cachaça Salivana 600 ML GP
Cachaça Segredo de Salinas
Cachaça Memoria de Salinas
Cachaça Amada Prata 700 ml
Dom Bré Tradicional 700 ml
Cachaça Barril 12 - 700 ml
Cachaça Nêga Fulô Carvalho
C

In [21]:
string_NomesBebidas = '|'.join(map(str, lista_Com_NomesDeBebidas_Ordenados))
string_NomesBebidas

"Vanderley Azevedo 12 anos + Vanderley Azevedo 6 anos + Vanderley Azevedo Prata + Vanderley Azevedo 4 anos _ Empório Cachaça Canela-de-Ema|Cachaça Artesanal Mineira João Plenário Blend 700 ml - Cachaça é Presente - Caipirinha Morena|Cachaça Havana Edição Comemorativa 75 anos 750 ml com Certificado N. 253 Tradicional|Cachaça Cambéba Extra Premium - 7 anos -  700 ml  _ Empório Cachaça Canela-de-Ema|Cachaça Espírito de Minas Coleção Artistas Brasileiros Por Elifas Andreato 700ml|Cachaça Trindade Artesanal Platinum Amendoim Porcelana 700 ml cachaça é Presente|Cachaça Mazzaropi Envelhecida Carvalho Francês 4 Anos 700 ml Cachaça e Presente|Cachaça Cambéba Extra Premium - 5 anos - 700ml  _ Empório Cachaça Canela-de-Ema|Cachaça Espírito de Minas Coleção Artistas Brasileiros Por Claudio Tozzi 700ml|Cachaça Rio do Engenho Reserva Umburana, Itiúba, Louro Canela e Balsamo 670 ml|Cachaça Casa Bucco Extra Premium 6 Anos Garrafa Especial Personalizada 700ml|Cachaça Oriente de Minas - Carvalho  - 670m

### Código Principal de rotulação semiautomatica

In [ ]:
def rotulacao_Entidades(dados_De_Texto_Para_Rotular): 
    t = []#list Final
    
    #Itera por cada linha da lista com texto a serem rotulados
    for i in dados_De_Texto_Para_Rotular:
        texto = i;
        listaL = [] #list Support
        print(texto)
        
        resultado_Comparacao = re.finditer(string_NomesBebidas, texto, re.I)

        #Itera pelos objetos resultantes do match encontrado no texto para a entidade da lista de entidades em questão  
        for index, iterator in enumerate(resultado_Comparacao): 
            #adiciona da lista 
            listaL.append((iterator.start(), iterator.end()+1, dicionario_EntidadeCategoria[iterator.group()]))
            print(iterator)
            print(listaL)
            
        if listaL:
            t.append((texto,{'entities':listaL}))
            #print('Lista T:',t,'\n')
        else:
            t.append((texto, {'entities':listaL}))
    return t            

In [39]:
def rotulacao_Entidades(dados_De_Texto_Para_Rotular): 
    t = []#list Final
    
    #Itera por cada linha da lista com texto a serem rotulados
    for i in dados_De_Texto_Para_Rotular:
        texto = i;
        listaL = [] #list Support
        print(texto)
        
        # Atribui a variavel 'resultado_Comparacao' um objeto do tipo 'callable_iterator', o qual contem informações dos matchs
        # ou seja, onde a entidade encontrado no texto inicia e finaliza bem como o grupo ao qual cada match pertence.
        resultado_Comparacao = re.finditer(string_NomesBebidas, texto, re.I)
        #print('match:', resultado_Comparacao)

        #Itera pelos objetos resultantes do match encontrado no texto para a entidade da lista de entidades em questão  
        for index, iterator in enumerate(resultado_Comparacao): 
            #adiciona da lista 
            listaL.append((iterator.start(), iterator.end()+1, dicionario_EntidadeCategoria[iterator.group()]))
            print(iterator)
            print(listaL)
            
        if listaL:
            t.append((texto,{'entities':listaL}))
            #print('Lista T:',t,'\n')
        else:
            t.append((texto, {'entities':listaL}))
    return t            

In [40]:
# Atribuindo o resultado da função a uma variavel 

textos_Rotulados = rotulacao_Entidades(dados_De_Texto_Para_Rotular)

id: 0
NOME DA CACHAÇA: Cachaça Três Coronéis e Cachaça Três Coronéis
<re.Match object; span=(17, 38), match='Cachaça Três Coronéis'>
[(17, 39, 'NOME_CACHACA')]
<re.Match object; span=(41, 62), match='Cachaça Três Coronéis'>
[(17, 39, 'NOME_CACHACA'), (41, 63, 'NOME_CACHACA')]
PREÇO: R$ 35,00
DESCRIÇÃO DA CACHAÇA: Cachaça Três Coronéis 700ml
<re.Match object; span=(22, 43), match='Cachaça Três Coronéis'>
[(22, 44, 'NOME_CACHACA')]
Volume: 750ml
Graduação Alcoólica: 39,2%
RECIPIENTE DE ARMAZENAMENTO: Tipo Barril: Carvalho
Tipo: Envelhecida
Produtor: Glauri Ind Com. Ltda
Local de Produção: Boituva / São Paulo
id: 1
NOME DA CACHAÇA: Cachaça Princesa Isabel Jaqueira Ouro
<re.Match object; span=(17, 54), match='Cachaça Princesa Isabel Jaqueira Ouro'>
[(17, 55, 'NOME_CACHACA')]
PREÇO: R$ 85,00
DESCRIÇÃO DA CACHAÇA: Na beira do Rio Doce cultivamos cacau na forma mais tradicional e sustentável.
Lá, um certo dia no ano de 2015, uma frondosa jaqueira secular tombou durante um temporal, foi quando

KeyError: 'Cachaça colombina 3 anos'

In [20]:
# Verificar o resultado do algoritmo de rotrulação, tanto o tamanho quando os dados gerados

textos_Rotulados

[('id: 0', {'entities': []}),
 ('NOME DA CACHAÇA: Cachaça Três Coronéis',
  {'entities': [(17, 39, 'NOME_CACHACA')]}),
 ('PREÇO: R$ 35,00', {'entities': []}),
 ('DESCRIÇÃO DA CACHAÇA: Cachaça Três Coronéis 700ml',
  {'entities': [(22, 44, 'NOME_CACHACA')]}),
 ('Volume: 750ml', {'entities': []}),
 ('Graduação Alcoólica: 39,2%', {'entities': []}),
 ('RECIPIENTE DE ARMAZENAMENTO: Tipo Barril: Carvalho', {'entities': []}),
 ('Tipo: Envelhecida', {'entities': []}),
 ('Produtor: Glauri Ind Com. Ltda', {'entities': []}),
 ('Local de Produção: Boituva / São Paulo', {'entities': []}),
 ('id: 1', {'entities': []}),
 ('NOME DA CACHAÇA: Cachaça Princesa Isabel Jaqueira Ouro',
  {'entities': [(17, 55, 'NOME_CACHACA')]}),
 ('PREÇO: R$ 85,00', {'entities': []}),
 ('DESCRIÇÃO DA CACHAÇA: Na beira do Rio Doce cultivamos cacau na forma mais tradicional e sustentável.',
  {'entities': []}),
 ('Lá, um certo dia no ano de 2015, uma frondosa jaqueira secular tombou durante um temporal, foi quando nosso patr

### Salvando os as tuplas compostas pelo texto e pelo dicionario para utilizar no treinamento do modelo de REN

In [21]:
with open('C:\\Users\\prisc\\Downloads\\BaseResultadoRotulacaoSemiautomatica', "wb") as fp:
    
    #for tupla in textos_Rotulados:
        #print(tupla)
        pickle.dump(textos_Rotulados,fp)

In [123]:
# Testando para ver se é possivel abrir o arquivo salvo com uma lista de tuplas

with open('C:\\Users\\prisc\\Downloads\\Algoritmo_Criacao_Automatica_Corpus_Treino\\codigoPrincipal\\Dados_Para_Treinar_o_Modelo_De_Reconhecimento_De_Entidades\\Dados_Rotulados_Com_Categoria_RecipienteArmazenamento_TipoMadeira_TreinarModelo.txt', "rb") as fp:
    listas_De_Tuplas = pickle.load(fp)
    #print(listas_De_Tuplas)
    
#listas_De_Tuplas

#### Para verificar se a lista possui itens repetidos